In [96]:
from officelib.xllib import *
from officelib.const import xlconst as xlc
import os
xl = Excel()

for rf in xl.RecentFiles:
    if os.path.basename(rf.Name) == "release notes test 3.0 categorize.xlsx":
        wb = rf.Open()
        break
else:
    raise NameError("Couldn't find the thing")
#wb = xl.Workbooks("release notes test 2 categorize.xlsx")
ws = wb.Worksheets(1)
cells = ws.Cells
cell_range = cells.Range
cr = cell_range

In [125]:
c1 = cr2("D1")
while c1.Value:
    ind = len(c1.Value)-1
    ind = ind * 2
    c1.Offset(1,2).IndentLevel = ind
    c1 = c1.Offset(2,1)

c1 = cr2("D1")
while c1.Value:
    stars = len(c1.Value)
    if stars == 1:
        c1.EntireRow.Insert()
    c1=c1.Offset(2,1)

In [97]:
# remove italics
for cell in xl.Selection:
    if not cell.Value:
        print(cell.Address)
        break
    cell.Value = cell.Value.replace("_", "")

$D$237


In [126]:
c = cr2("D2")
while True:
    if c.Value is None:
        break
    c2 = c.Offset(2,1)
    if c2.Value is None:
        c2.EntireRow.Delete()
        if c.Offset(2,1).Value is None:
            break
    else:
        c = c.Offset(2,1)

In [128]:
from collections import OrderedDict

cat_map = {
    "webui": "Hello UI",
    "bugfix": "Bug Fixes",
    "misc": "Miscellaneous",
    "other": "Miscellaneous",
    "gmp": "GMP and CFR 21 part 11 Compliance",
    "shell": "Desktop UI",
    "harvest": "Harvest Mode (15L and greater only)",
    "controls": "Controls",
    "permissions": "Permissions",
    "data_and_reports": "Data and Reports",
    "decaytest": "Pressure Decay Test (15L and greater only)",
    "security": "Security",
    "alarms": "Alarms",
    'improvements': "Improvements",
    "bug": "Bug Fixes",
    "exclude": "Exclude",
    "summary": "Summary"
    
}

def add_note(cat1, cat2, issue, indent, note):
    cat1 = cat_map.get(cat1, cat1.title())
    cat2 = cat_map.get(cat2, cat2.title()) if cat2 is not None else None
    if cat1 not in notesd:
        notesd[cat1] = OrderedDict()
    l1 = notesd[cat1]
    if cat2 not in l1:
        l1[cat2] = []
    l2 = l1[cat2]
    l2.append((indent, note))
    
c1 = cr2("A2")
notesd = OrderedDict()
data = cr2(c1, c1.End(xlc.xlDown).Offset(1,5)).Value
    
for cat1, cat2, issue, indent, notes in data:
    add_note(cat1, cat2, issue, indent, notes)
    

In [103]:
def text_out(notesd, fn):
    out = []
    summary = notesd['Summary'][None]
    out.append("Summary:")
    for ind, note in summary:
        out.append("".join(("    ", ind, " ", note)))
        
    for cat1, l1 in notesd.items():
        if cat1 in ("Summary", "Exclude"):
            continue
        out.append("\n%s: "%cat1)
        for cat2, l2 in l1.items():
            if cat2 is not None:
                pre = "        "
                out.append("    %s:"%cat2)
            else:
                pre = "    "
            for indent, note in l2:
                out.append(''.join((pre, (len(indent)-1)*"    ",indent," ",note)))

    with open(fn, 'w') as f:
        f.write("\n".join(out))
text_out(notesd, "notes test2.txt")

In [104]:
%load_ext autoreload
%autoreload 2
from officelib.wordlib import Word
from officelib.wordlib import c as wdc
from officelib import wordlib
from pythoncom import com_error
from officelib import const as wdc

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [105]:
def make_style(doc, name="PY_RN_TITLE", font_size=24, font="Calibri", 
                     alignment=wdc.wdAlignParagraphLeft, indent=None, tabstops=None, space_after=None):
    
    if alignment is None:
        alignment = wordlib.c.wdAlignParagraphCenter
    
    style = doc.Styles.Add(name)
    style.BaseStyle = doc.Styles("Normal")
    style.NoSpaceBetweenParagraphsOfSameStyle = True
    
    style.Font.Size = font_size
    style.Font.Name = font
    
    style.ParagraphFormat.Alignment = alignment
    style.ParagraphFormat.SpaceBeforeAuto = False
    style.ParagraphFormat.SpaceAfterAuto = False
    style.ParagraphFormat.LineSpacingRule = wordlib.c.wdLineSpaceSingle
    
    if indent is not None:
        style.ParagraphFormat.LeftIndent = wordlib.inches_to_points(indent)
    
    if tabstops is not None:
        style.ParagraphFormat.TabStops.Add(wordlib.inches_to_points(tabstops), 
                                           wordlib.c.wdAlignTabLeft, 
                                           wordlib.c.wdTabLeaderDots)
    
    if space_after is not None:
        style.ParagraphFormat.SpaceAfter = space_after
    
    return style

In [129]:
_styles = {}
def get_style(d, indent, cat):
    n = len(indent)
    i = n
    if cat is not None:
        i += 1
    name = "PY_RN_BULLET%d_%d"%(i, n)
    style = _styles.get(name)
    if style is None:
        style = bulletstyle(d, name, n, 9)
        _styles[name] = style
    return style

def move(r,n):
    r.MoveStart(wdc.wdCharacter, n)
    
def pwrite(r, txt, style):
    if isinstance(style, str):
        style = _styles[style]
    r.Text = txt
    r.Style = style
    r.InsertAfter("\r")
    move(r, len(txt)+1)
    
def _unpack_styles(d):
    for style in d.Styles:
        _styles[style.NameLocal] = style
            
def _normal_style(d):
    style = make_style(d, "PY_RN_NORMAL", 9, 0)
    _styles[style.NameLocal] = style
    
def clear(d):
    r = d.Paragraphs(1).Range
    r.MoveEnd(wdc.wdStory, 1)
    r.Text = ""
    r.Style = "Normal"
    
def word_out(w, d, notesd):
    clear(d)
    _unpack_styles(d)
    pgs = d.Paragraphs
    r = pgs(1).Range
    
    # summary first
    l2 = notesd["Summary"]
    pwrite(r, "%s:"%"Summary", "PY_RN_CAT1")
    for _, lines in l2.items():
        for _, line in lines:
            pwrite(r, line, "PY_RN_BULLET_1_0")
    
    # all items except 'Summary' and 'Exclude'
    for cat1, l2 in notesd.items():
        if cat1 == "Summary" or cat1 == "Exclude":
            continue
        pwrite(r, "%s:"%cat1, 'PY_RN_CAT1')
        for cat2, l2 in l2.items():
            if cat2 is not None:
                pwrite(r, "%s:"%cat2, 'PY_RN_CAT2')
            for ind, note in l2:
                n = len(ind)
                i = int(bool(cat2))
                pwrite(r,note, 'PY_RN_BULLET_%d_%d'%(n,i))
    

In [130]:
w = Word()
for rf in w.RecentFiles:
    if rf.Name.endswith("Release Notes Template5.docx"):
        d = w.Documents.Open(rf.Path + "\\" + rf.Name, ReadOnly=True)
        break
else:
    raise ValueError
#clear(d)

In [131]:
with wordlib.screen_lock(w):
    word_out(w,d,notesd)

In [122]:
try:
    ws2 = wb.Worksheets("Sheet1")
except com_error:
    ws2 = wb.Worksheets.Add()
    
cr2 = ws2.Cells.Range
c2_1 = cr2("A1")
c1 = cr("A2")

def writerow(*args):
    global c2_1
    cr2(c2_1, c2_1.Offset(1, len(args))).Value = [args]
    c2_1 = c2_1.Offset(2,1)
    
data = cr(c1, c1.End(xlc.xlDown).Offset(1,5)).Value
cats = {(cat1, cat2) for cat1, cat2, _, _, _ in data}
order = "summary shell webui gmp controls alarms harvest misc bugfix exclude".split()
order2 = [cat_map[o] for o in order]
set(cat_map[c[0]] for c in cats)-set(order2)
for o in order2:
    for c1, c2, i, ind, note in data:
        if cat_map[c1] == o:
            writerow(c1, c2, i, ind, note)

In [118]:
ws2.Name

'release notes test 3.0 categori'

In [ ]:
cat_map